# Valiant Neuroidal Model Simulation
by Patrick Perrine and Chandradeep Chowdhury

Published with the submission for Rwebangira, M., Perrine, P., and Chowdhury C. 
(2023, August). Capacity in Valiant’s Neuroidal Model for Shared Memory 
Representations. *Conference on Cognitive Computational Neuroscience 2023*.


### Install the *graph-tool* library (https://graph-tool.skewed.de/)

In [1]:
%%capture
!echo "deb http://downloads.skewed.de/apt focal main" >> /etc/apt/sources.list
!apt-key adv --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
!apt-get update
!apt-get install python3-graph-tool python3-matplotlib python3-cairo

### Import required libraries and seed random number generators

In [2]:
%%capture
import gc, os, sys 
import numpy as np
from numpy.random import *
import graph_tool as gt

seed(42)
np.random.seed(42)
gt.seed_rng(42)

### Define the neuroidal models' properties
The following configuration was used to create the leftmost datapoints of the 
shared representation capacity in Figure 1 of Rwebangira et al. (2023).

* The first six parameters are as described by Valiant (2005). 
* The *H* parameter determines how many memories to JOIN in one "batch" of 
simulation. 
* The *STOP* parameter determines the level of interference tolerance to be 
checked at each iteration of JOIN. 
* The *START_MEM* parameter specifies how many initial memories to grant the 
model before calling JOIN. 
* The *r_expected* parameter is used to generate the random, initial memories 
in a size as expected by the relations defined in Valiant (2005).

In [3]:
N = 500
D = 128
T = 1
k = 16
k_adj = 1.55 
P = D / (N - 1)

H = 100 
STOP = 0.25
START_MEM = 100
r_expected = 40 

## Generate the graph model

### Create an empty, directed graph

In [4]:
g = gt.Graph()

### Populate our graph with $N$ vertices

In [5]:
%%capture
g.add_vertex(N)

### Define the graph's vertices to have properties

In [6]:
vprop_fired = g.new_vertex_property("int")
vprop_memories = g.new_vertex_property("int")
vprop_fired_now = g.new_vertex_property("int")
vprop_weight = g.new_vertex_property("double")
vprop_threshold = g.new_vertex_property("double")

### Initialize the properties of each vertex

In [7]:
vprop_fired.a = 0
vprop_memories.a = 0
vprop_fired_now.a = 0
vprop_weight.a = 0.0
vprop_threshold.a = T

### Calculate all possible edges
While computationally costly, this procedure helps us ensure that we are 
properly generating an Erdős–Rényi $G=(n,p)$ random graph.

In [8]:
x, y = np.meshgrid(np.arange(N), np.arange(N))
mask = x != y
x = x[mask]
y = y[mask]
pairs = np.stack((x, y), axis=1)

### Conduct $N^2 - N$ Bernoulli trials to calculate the number of edges

In [9]:
z = np.random.default_rng().geometric(p=P, size=((N*N)-N))
num_edges = (z == 1).sum()

print("Number of edges determined: ", num_edges)

Number of edges determined:  63658


### Memory manage the large Bernoulli distribution used

In [10]:
%%capture
del z
gc.collect()

### Randomly choose those edges using our previously simulated number of edges

In [11]:
index = np.random.default_rng().choice(pairs.shape[0], 
                                       size=int(num_edges), 
                                       replace=False)

### Add list of edges to the graph

In [12]:
g.add_edge_list(pairs[index])

### Memory manage the lists of all possible vertex combinations

In [13]:
%%capture
del x, y
gc.collect()

### Define the graph's edges to have properties

In [14]:
eprop_fired = g.new_edge_property("int")
eprop_weight = g.new_edge_property("double")

### Initialize the properties of each edge

In [15]:
eprop_fired.a = 0
eprop_weight.a = (T / (k_adj * k))

## Vicinal Algorithms for the JOIN algorithm

### Memory trace and creation function

In [16]:
def check_and_fire_and_add(v, memory_C):
  sum = 0
  for s,t in g.iter_in_edges(v):
    if vprop_fired_now[s] > 0:
        sum += eprop_weight[g.edge(s,t)]
  if sum > vprop_threshold[v]:
    vprop_fired[v] += 1
    memory_C.append(v)

### Interference check function

In [17]:
def interference_check(g, memory_bank, a, b, memory_C):
  sum = 0
  for i in range(len(memory_bank)):
    if i != a and i != b: 
      inter = list(set(memory_C) & set(memory_bank[i]))
      if len(inter) > len(memory_bank[i]) / 2:
        sum += 2
  return sum

## The JOIN algorithm
This implements the *one-step* variant of JOIN for *shared representations* as 
defined in Valiant (2005).

In [18]:
def JOIN_shared_one_step(g, memory_bank, i, j):
  """
    Choose two random groups of neurons to become A and B
      Basing this on the expected value of r from Valiant (2005)
    Set A, then B to fire
    Trace C from the firing nodes outward from A and B
    Check for interference
  """

  memory_A = memory_bank[i]
  memory_B = memory_bank[j]

  #Fire A
  for v in memory_A:
    vprop_fired_now[v] = 1
    vprop_fired[v] += 1
    vprop_memories[v] += 1

  #Fire B
  for v in memory_B:
    vprop_fired_now[v] = 1
    vprop_fired[v] += 1
    vprop_memories[v] += 1

  memory_C = []
  #Check and fire adjacent nodes:
  for v in g.iter_vertices():
    check_and_fire_and_add(v, memory_C)

  inter = interference_check(g, memory_bank, i, j, memory_C)
  vprop_fired.a = 0
  vprop_fired_now.a = 0
  memory_bank.append(memory_C)

  return inter, len(memory_C)

## Simulation



### Generate the random, initial memories
This is simply noise for which the JOIN algorithm can use so we may observe the
algorithm's execution in action. These initial memories are not intended to 
represent any real "information" for which the model is meant to memorize. We 
continue to operate on the "blank slate" notion for the implementation of this 
model.

It is important that the number of starting memories does not exceed the 
model's capacity, as the resulting execution of JOIN will be halted. It is also 
important that there are not too few starting memories, as the model can end up 
"memorizing" everything possible from these memories.

In [19]:
memory_bank = []
for i in np.arange(0, START_MEM):
  memory_A = np.random.default_rng().choice(np.arange(0,N-1), size=r_expected)
  gc.collect()
  memory_bank.append(memory_A)

i, j = np.meshgrid(np.arange(len(memory_bank)), np.arange(len(memory_bank)))
mask = i != j
i = i[mask]
j = j[mask]
gc.collect()
pairs = np.unique(np.sort(np.stack((i,j), axis = 1)), axis=0)
np.random.shuffle(pairs)

### Call JOIN on all possible pairs of memories until the interference threshold is reached
Assuming that the threshold will be reached and this code's execution will 
halt, we will have reached the model's final capacity.

In [20]:
total_inters = 0
ind = 0
inst_inters = 0
inst_len = 0

for pair in pairs:
    ind += 1
    i = pair[0]
    j = pair[1]
    inter_flag, length = JOIN_shared_one_step(g, memory_bank, i, j)
    inst_len += length
    if ind % H == 0:
      print("Memories: ", len(memory_bank))
      print("Instantaneous interference rate: ", inst_inters/H)
      print("Average interference rate: ", total_inters/len(memory_bank))
      print("Average size of memories created: ", inst_len/H, "\n\n")
      inst_inters = 0
      inst_len = 0
    if inter_flag > 0:
      total_inters += inter_flag
      inst_inters += inter_flag
      if total_inters/len(memory_bank) > STOP:
        print("Config: N=", N, " D=",D, " k=", k, " k_adj=", k_adj, " R=", 
              r_expected, "START_MEM=", START_MEM)
        print("Halting memory formation at ", len(memory_bank), 
              " memories due to more than ", STOP*100, 
              "percent total interference")
        print("Instantaneous interference rate: ", inst_inters/H)
        print("Average interference rate: ", total_inters/len(memory_bank))
        break

Memories:  200
Instantaneous interference rate:  0.2
Average interference rate:  0.1
Average size of memories created:  35.39 


Config: N= 500  D= 128  k= 16  k_adj= 1.55  R= 40 START_MEM= 100
Halting memory formation at  255  memories due to more than  25.0 percent total interference
Instantaneous interference rate:  0.44
Average interference rate:  0.25098039215686274


### Comparison with the exact capacity for the *disjoint representation* model

In [21]:
print("Exact capacity in disjoint representation:", (int)(N / r_expected))

Exact capacity in disjoint representation: 12
